#Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
os.listdir('/content/drive/My Drive/refrigerator1/')#path of refrigerator data in blueteam drive

['put_back_item',
 'no_interaction',
 'open_close_fridge',
 'screen_interaction',
 'take_out_item',
 'full_features',
 'test']

#Interpolation

In [ ]:
def traj_interp(traj, n_frames=100):
    n_samples = traj.shape[0]

    if n_samples == 0:
        raise ValueError("trajectories of length 0!!")

    result = np.empty((n_frames, 2, 17))

    traj = np.asarray(traj)
    dest_x = np.linspace(0, 100, n_frames)
    src_x = np.linspace(0, 100, n_samples)

    for i in range(2):
        for j in range(17):
            result[:, i, j] = np.interp(
                dest_x,
                src_x,
                traj[:, i, j]
            )

    return result.reshape(-1)

In [ ]:
def extract_trajectories(keypoints, with_index=False):
    """
    with_index: For True, return the index of frames that have features
    """
    trajectory = []
    index = []
    for i, (_, k) in enumerate(keypoints):
        if len(k) != 0:
            index.append(i)
            two_d_point = k[0, [0, 1], :]
            trajectory.append(two_d_point)

    if with_index:
        return np.stack(trajectory), index
    else:
        return np.stack(trajectory)

In [ ]:
def get_full_feature_data(feature_dirs, transform=None, **kwargs):
    features = []
    labels = []
    label_encoder = {'no_interaction': 0, 'open_close_fridge': 1,
                     'put_back_item': 2, 'screen_interaction': 3, 'take_out_item': 4}

    for path in feature_dirs:
        #label = path.split('/')[-2]
        name=path.split('/')[-1].split('_')[:-3]
        if len(name)==2:
          label=name[0]+'_'+name[1]
        else:
          label=name[0]+'_'+name[1]+'_'+name[2]

        d = np.load(path, allow_pickle=True)
        traj = extract_trajectories(d['keypoints'], with_index=False)
        if transform:
            traj = transform(traj, **kwargs)
        labels.append(label_encoder.get(label, None))
        features.append(traj)

    return features, np.stack(labels)

In [ ]:
from glob import glob
path_features = '/content/drive/My Drive/refrigerator1/full_features/outputs'
#load both mp4 and mkv file
npz_list = [y for x in os.walk(path_features) for y in glob(os.path.join(x[0], '*.mp4.npz'))]+[y for x in os.walk(path_features) for y in glob(os.path.join(x[0], '*.mkv.npz'))]
X, y = get_full_feature_data(npz_list, transform = traj_interp)
X = np.asarray(X)

In [ ]:
print(X.shape)
print(y.shape)

(440, 3400)
(440,)
